In [4]:
import pandas as pd
import glob
import numpy as np
xls_file_list = glob.glob(r'.\*.xls')
df = []
for i, f in enumerate(xls_file_list):
    xls = pd.read_excel(f)
    xls = xls.drop(['SACCESSIONNO', 'STYPE', 'SCREATEDATETIME','SPATIENTID', 'XMLID', 'CREATEDATETIME'],
                   axis = 1)
    df += [xls]

df = pd.concat(df,axis=0, ignore_index = True)


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (13740617) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (12265151) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (4930923) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [5]:
def xml_parser(xml_data):

    xml_data = xml_data.replace('\n','').replace(' ','').split('><')

    data = '\n'
    for j, temp in enumerate(xml_data):
        len_ascii = []
        for k in temp:
            len_ascii += [len(ascii(k))]
        if '?' in temp and j > 0:
            xml_data[j] = ''
        elif j != len(xml_data) -1 and j != 0:
             xml_data[j] = '<' + xml_data[j] + '>'
        elif j == len(xml_data) -1:
            xml_data[j] = '<' + xml_data[j]
        elif j == 0:
            xml_data[j] = '<?xml version="1.0" encoding="UTF-8"?>'

            
    
    xml_data = data.join(xml_data)
# 
    root = ET.fromstring(xml_data)
    elements = root.iter()
    dict = {}
    for element in elements:
        dict[element.tag] = element.text
    return dict

In [ ]:

for i in df['XML'][0]:
    print(i,' : ',i.encode(),' : ',ascii(i),' : ',len(ascii(i)))

#     try:
#         i.encode('cp950').decode('big5')
#         print('try _ 1 : ' + i)
#     except:
#         try:
#             i.encode('gbk').decode('big5')
#             print('not - 1 : ' + i)
#         except:
#             try:
#                 i.encode('big5').decode('gbk')
#                 print('not - 2 : ' + i)
#             except:
#                 i.encode('utf-8').decode('utf-8')
#                 print('not - 3 : ' + i)
print(df['XML'][0])

In [7]:
import xml.etree.ElementTree as ET
import re
import string

filtrate = string.printable.replace('\n\n','\n').replace('\n\n\n','\n')
filtrate = re.compile("[^"+ filtrate +"]")
df['xml_data'] = df['XML'].replace(filtrate,'').apply(xml_parser)
# df.index = list(range(len(df)))

In [8]:

# for tag_dict in df['xml_data']:
output = pd.concat([pd.DataFrame(tag_dict, index=[0]) for i, tag_dict in enumerate(df['xml_data'])], axis = 0, sort=False, ignore_index = True)

# output.index = list(range(len(output)))

In [ ]:
output = output.replace('\n',float('nan')).dropna(axis = 'columns', how = 'all')

In [ ]:
output.to_excel("./output/output.xls")

In [ ]:
output